In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import join
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, History, TensorBoard, ModelCheckpoint

In [ ]:
%load_ext tensorboard

In [ ]:
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

%tensorboard --logdir {logs_base_dir}

In [3]:
folder_weights_name = "save_weights"
checkpoint_path = join(folder_weights_name, 'weights')

checkpoint_dir = os.path.dirname(checkpoint_path)

In [4]:
# To load the weights and evaluate the performace from a previous model
# we need to generate a new model instance
model = model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# Load the previously saved weights
model.load_weights(latest)

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

W1108 19:00:50.987185 4638746048 base.py:272] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0xb33710c18> and <tensorflow.python.keras.layers.core.Dropout object at 0xb33710b00>).


save_weights/weights


In [ ]:
model.weights[0].shape

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

loss, acc = model.evaluate(x_test,  y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/1 - 1s - loss: 1.1444 - accuracy: 0.5402
Restored model, accuracy: 54.02%


In [ ]:
model.summary()

In [ ]:
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
!ls {checkpoint_dir}

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# [43941 16565  3582 16743 21846 13094 36857 51583 47753  1103]
# [[ 6394, 10985, 26146, 24512, 47211, 54289, 34046, 13024, 48617, 8563]
# [27894, 41222, 37632,  2735, 42382,  9800, 32978,  5897, 32480, 45969]

# 51737, 10259, 43404
#15253,  3925, 50169

In [ ]:
plt.imshow(x_train[50169])

In [ ]:
def crop(X, edge):
    """Here we pass the data matrix, the tensor of dimensions (#n_samples, dim1, dim2) """
    _, dim1, dim2 = X.shape
    X_ = X[:, edge:dim1-edge, edge:dim2-edge]
    return X_

In [ ]:
for e_ in [2, 4, 6, 8, 9, 10]:
    xx = crop(x_train, e_)
    plt.imshow(xx[3])
    plt.show()
xx.shape

In [ ]:
n_, dim1, dim2 = xx.shape
xx = np.reshape(xx, (n_, -1), order='C')

In [ ]:
def crop(X, edge=3):
    """ Here we pass the data matrix, the tensor of dimensions (#n_samples, dim1, dim2) """
    _, dim1, dim2 = X.shape
    X_ = X[:, edge:dim1-edge, edge:dim2-edge]
    return X_


def pyramid(X, edge_list=[0, 2, 4, 6, 8, 9, 10]):
    """ Here we make the pyramid, meaning that we zoom into the data """
    n_samples, dim1, dim2 = X.shape
    for i_, e_ in enumerate(edge_list):
        if i_ == 0:
            X_ = np.reshape(crop(X, e_), (n_samples, -1), order='C')
        else:
            X_ = np.hstack((X_, np.reshape(crop(X, e_), (n_samples,-1), order='C')))
    return np.reshape(X_, (n_samples, X_.shape[1], 1))

In [ ]:
xxx = pyramid(x_train, edge_list=[4,8,10])

In [ ]:
x_train[0][4:24, 4:24].shape

In [ ]:
plt.imshow(x_train[0][4:24, 4:24])

In [ ]:
plt.imshow(xxx[0, 20**2:20**2+12**2].reshape(12,12))

In [ ]:
edge_list = np.arange(5, 50, 5)
edge_list

In [ ]:
n_training = np.load('example_results/n_training_array.npy')
test_acc = np.load('example_results/original/test_acc.npy')
test_loss = np.load('example_results/original/test_loss.npy')

mean_acc = test_acc.mean(axis=0)
std_acc = test_acc.std(axis=0)
plt.fill_between(n_training, mean_acc-std_acc, mean_acc+std_acc, alpha=0.2)
plt.plot(n_training, mean_acc, 'o-', linewidth=3)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test accuracy', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.show();

mean_loss = test_loss.mean(axis=0)
std_loss = test_loss.std(axis=0)
n_training = np.load('example_results/n_training_array.npy')
plt.fill_between(n_training, mean_loss-std_loss, mean_loss+std_loss, alpha=0.2)
plt.plot(n_training, mean_loss, 'o-', linewidth=3)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test loss', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);

In [ ]:
n_training = np.load('example_results/n_training_array.npy')
test_acc_crop = np.load('example_results/crop/test_acc.npy')
test_loss_crop = np.load('example_results/crop/test_loss.npy')
mean_acc_crop = test_acc_crop.mean(axis=0)
std_acc_crop = test_acc_crop.std(axis=0)

test_acc = np.load('example_results/original/test_acc.npy')
test_loss = np.load('example_results/original/test_loss.npy')
mean_acc = test_acc.mean(axis=0)
std_acc = test_acc.std(axis=0)

plt.fill_between(n_training, mean_acc-std_acc, mean_acc+std_acc, alpha=0.2, color='C0')
plt.plot(n_training, mean_acc, 'o-', linewidth=3, color='C0', label='original')

plt.fill_between(n_training, mean_acc_crop-std_acc_crop, 
                 mean_acc_crop+std_acc_crop, alpha=0.2, color='C1')
plt.plot(n_training, mean_acc_crop, 'o-', linewidth=3, color='C1', label='cropped')
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test accuracy', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.show();

mean_loss = test_loss.mean(axis=0)
std_loss = test_loss.std(axis=0)
n_training = np.load('example_results/n_training_array.npy')
plt.fill_between(n_training, mean_loss-std_loss, mean_loss+std_loss, alpha=0.2, color='C0')
plt.plot(n_training, mean_loss, 'o-', linewidth=3, color='C0', label='original')
plt.fill_between(n_training, mean_loss_crop-std_loss_crop, 
                 mean_loss_crop+std_loss_crop, alpha=0.2, color='C1')
plt.plot(n_training, mean_loss_crop, 'o-', linewidth=3, color='C1', label='cropped')
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test loss', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.show();

In [ ]:
n_training = np.load('example_results/n_training_array.npy')
test_acc_pyr = np.load('example_results/pyramid_all_layers/test_acc.npy')
test_loss_pyr = np.load('example_results/pyramid_all_layers/test_loss.npy')
mean_acc_pyr = test_acc_pyr.mean(axis=0)
std_acc_pyr = test_acc_pyr.std(axis=0)

plt.fill_between(n_training, mean_acc-std_acc, 
                 mean_acc+std_acc, alpha=0.2, color='C0')
plt.plot(n_training, mean_acc, 'o-', linewidth=3, color='C0', label='original')
plt.fill_between(n_training, mean_acc_pyr-std_acc_pyr, 
                 mean_acc_pyr+std_acc_pyr, alpha=0.2, color='C1')
plt.plot(n_training, mean_acc_pyr, 'o-', linewidth=3, color='C1', label='inverted-pyr')

plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test accuracy', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.title('pyramid w all crops', fontsize='xx-large')
plt.tight_layout()
plt.savefig('pyramid_acc.pdf')
plt.show();

mean_loss_pyr = test_loss_pyr.mean(axis=0)
std_loss_pyr = test_loss_pyr.std(axis=0)
n_training = np.load('example_results/n_training_array.npy')
plt.fill_between(n_training, mean_loss-std_loss, mean_loss+std_loss, alpha=0.2, color='C0')
plt.plot(n_training, mean_loss, 'o-', linewidth=3, color='C0', label='original')
plt.fill_between(n_training, mean_loss_pyr-std_loss_pyr, 
                 mean_loss_pyr+std_loss_pyr, alpha=0.2, color='C1')
plt.plot(n_training, mean_loss_pyr, 'o-', linewidth=3, color='C1', label='inverted-pyr')
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test loss', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.title('pyramid w all crops', fontsize='xx-large')
plt.tight_layout()
plt.savefig('pyramid_loss.pdf')

plt.show();

In [ ]:
test_acc_pyr = np.load('example_results/pyramid_zoomed_in/test_acc.npy')
test_loss_pyr = np.load('example_results/pyramid_zoomed_in/test_loss.npy')

plt.fill_between(n_training, mean_acc-std_acc, 
                 mean_acc+std_acc, alpha=0.2, color='C0')
plt.plot(n_training, mean_acc, 'o-', linewidth=3, color='C0', label='original')
plt.fill_between(n_training, mean_acc_pyr-std_acc_pyr, 
                 mean_acc_pyr+std_acc_pyr, alpha=0.2, color='C1')
plt.plot(n_training, mean_acc_pyr, 'o-', linewidth=3, color='C1', label='inverted-pyr')

plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test accuracy', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.title('pyramid w few crops', fontsize='xx-large')
plt.tight_layout()
plt.savefig('pyramid_acc.pdf')
plt.show();


mean_loss_pyr = test_loss_pyr.mean(axis=0)
std_loss_pyr = test_loss_pyr.std(axis=0)
n_training = np.load('example_results/n_training_array.npy')
plt.fill_between(n_training, mean_loss-std_loss, mean_loss+std_loss, alpha=0.2, color='C0')
plt.plot(n_training, mean_loss, 'o-', linewidth=3, color='C0', label='original')
plt.fill_between(n_training, mean_loss_pyr-std_loss_pyr, 
                 mean_loss_pyr+std_loss_pyr, alpha=0.2, color='C1')
plt.plot(n_training, mean_loss_pyr, 'o-', linewidth=3, color='C1', label='inverted-pyr')
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('test loss', fontsize='xx-large')
plt.xlabel('#training samples', fontsize='xx-large')
plt.grid()
plt.xlim([n_training[0]-0.1,n_training[-1]+0.1]);
plt.legend(fontsize='xx-large')
plt.title('pyramid w few crops', fontsize='xx-large')
plt.tight_layout()
plt.savefig('pyramid_loss_few_layers.pdf')

plt.show();

In [ ]:
np.append(0, np.cumsum(tmp))

In [ ]:
np.sum(tmp)

In [ ]:
ls

In [ ]:
?? np.reshape

In [ ]:
datafile = np.load(join('./red_square', 'x_test.npy'))
print(datafile.shape)

In [ ]:
dim = 28

In [ ]:
24**2

In [ ]:
dim_val = 0
ex = datafile[2]
for e_ in [0, 2, 4, 6, 8, 9, 10]:
    
    xx = np.reshape(a=ex[dim_val:dim_val+(dim-(2*e_))**2], newshape=(dim-(2*e_),dim-(2*e_)), order='C')
    dim_val += (dim-(2*e_))**2
    print((dim-(2*e_))**2)
    print(dim_val)
    plt.imshow(xx, vmin=0, vmax=1)
    plt.colorbar()
    plt.show()

In [ ]:
plt.imshow(x_test[0])

In [ ]:
plt.imshow(generate_bool_mask(x_test)[40])

In [ ]:
np.round(3.49).astype(int)

In [7]:
def upscale(x, e=2):
    n_samples, dim1, dim2 = x.shape
    c = 0
    ll = np.zeros(dim1, dtype=int)
    
    for k in range(dim1):
        ll[k] = np.round(c).astype(int)
        c += (dim1+2*e) / dim1
    min_diff = np.min(np.diff(ll))

    x_ = np.zeros((n_samples, dim1+2*e, dim2+2*e))
    dimyy, dimxx = np.meshgrid(ll, ll)
    x_[:, dimxx, dimyy] = x
    
    x__ = x_.copy()
    for n in range(n_samples):
        for i in range(min_diff, dim1+2*e - min_diff):
            for j in range(min_diff, dim2+2*e - min_diff):
                tmp_array = np.array([x_[n, i-min_diff:i+min_diff+1, 
                                        j-min_diff:j+min_diff+1]])
                tmp_non_zeros = np.count_nonzero(tmp_array)
                if tmp_non_zeros > 0:
                    x__[n, i, j] = np.sum(tmp_array) / tmp_non_zeros  # normalize in some way
                    
    return x__

# x_up = upscale(x_test)

edges = [int((ii-28)/2) for ii in [36, 42, 56, 80, 150]]
print(edges)
for e_ in edges:
    print(e_)
    x_up = upscale(x_test, e_)

[4, 7, 14, 26, 61]
4
7


KeyboardInterrupt: 

In [ ]:
plt.imshow(x_test[4])

In [ ]:
def upscale(x, e=4):
    n_samples, dim1, dim2 = x.shape
    c = 0
    ll = np.zeros(dim1, dtype=int)
    
    for k in range(dim1):
        ll[k] = np.round(c).astype(int)
        c += (dim1+2*e) / dim1
        
    print(np.max(np.diff(ll)), ll)

    x_ = np.zeros((n_samples, dim1+2*e, dim2+2*e))
    dimyy, dimxx = np.meshgrid(ll, ll)
    x_[:, dimxx, dimyy] = x
    
    x__ = x_.copy()
    return x__

plt.imshow(upscale(x_test[:3], e=10)[0])

In [ ]:
np.arange(10)[::2]